In [117]:
from time import *
from ephem import *
import math
import datetime
#
today=strftime("%Y/%m/%d")
horizon='20:00' # can also give in radians,

KAT=Observer()
KAT.long,KAT.lat='21:24:38.46','-30:43:17.34' # positive East,North
#KAT.date=today
#KAT.epoch=today
KAT.date='2017/05/20'
KAT.epoch=KAT.date
# print KAT.epoch
KAT.elev=1038            #-metres above MSL
KAT.temp=14             # temperature in C
KAT.pressure=0         # pressure in mbar set to zero to ignore refraction
KAT.horizon=horizon    # minimum elevation
dd=KAT.date.tuple()
print "Date %d/%d/%d UT %02d:%02d  LST %s Horizon %s"\
      %(dd[0],dd[1],dd[2],dd[3],dd[4],KAT.sidereal_time(),str(KAT.horizon))
print "Source\t\t  RA \t\t Dec \t  UTRise    UTTransit UTSet"
#
sun=Sun()
sun.compute(KAT)
#print "Sun  Alt",  sun.alt, "Az", sun.az
print "Sun \t\t%-12s %12s %-9s %-9s %-9s " \
      %(sun.ra,sun.dec,str(KAT.next_rising(sun))[-8:],\
        str(KAT.next_transit(sun))[-8:],str(KAT.next_setting(sun))[-8:])
#
moon=Moon()
moon.compute(KAT)
#print "Moon Alt", moon.alt,"Az", moon.az

print "Moon \t\t%-12s %12s %-9s %-9s %-9s " \
            % (moon.ra,moon.dec,str(KAT.next_rising(moon))[-8:],\
               str(KAT.next_transit(moon))[-8:],str(KAT.next_setting(moon))[-8:])




Date 2017/5/20 UT 00:00  LST 17:17:00.25 Horizon 20:00:00.0
Source		  RA 		 Dec 	  UTRise    UTTransit UTSet
Sun 		3:47:51.57     19:58:13.9 07:08:30  10:30:53  13:53:00  
Moon 		22:54:15.95    -8:00:25.5 00:52:04  05:45:13  10:34:24  


In [118]:
def SAST(utstring): #YYYY/MM/DD hh:mm:ss format
    ut=datetime.datetime.strptime(utstring,"%Y/%m/%d %H:%M:%S")
    sast=ut+datetime.timedelta(hours=2)
    sasts= sast.strftime("%Y/%m/%d %H:%M:%S") # with seconds
    return sasts
    

In [119]:
def UT(day,time):# format YYYY/MM/DD and hh:mm - ignore seconds
    sast=datetime.datetime.strptime(day+time,"%Y/%m/%d%H:%M") # datetime format
    ut=sast- datetime.timedelta(hours=2)
    UTS=ut.strftime("%Y/%m/%d %H:%M:%S") # neat UT string with seconds
    return UTS


In [120]:
def LST(ut):
    """ 
    Requires date and time in YYYY/MM/DD hh:mm:ss format
    Set the time string to Hours:Minutes:Seconds
    """
    KAT.date=ut
    lst=str(KAT.sidereal_time())[:8]
    return lst


In [121]:
def JD(ut):
    """ 
    Requires date and time in YYYY/MM/DD hh:mm:ss format
    returns jd
    """
    jd=julian_date(ut)
    return jd

In [122]:
local="13:45"+":00"
print local
datetime.time()


13:45:00


datetime.time(0, 0)

In [123]:
day="2017/05/19"   #string
time="00:15"       #string
sast=datetime.datetime.strptime(day+time,"%Y/%m/%d%H:%M") # datetime
print sast
ut=sast- datetime.timedelta(hours=2)
print ut
print "SAST",sast.strftime("%Y/%m/%d %H:%M:%S")," UT",ut.strftime("%Y/%m/%d %H:%M:%S")
                                


2017-05-19 00:15:00
2017-05-18 22:15:00
SAST 2017/05/19 00:15:00  UT 2017/05/18 22:15:00


In [124]:
print SAST("2017/05/13 22:46:00")

2017/05/14 00:46:00


In [125]:
x=UT("2017/05/16","13:00")
print x

2017/05/16 11:00:00


In [126]:
KAT.date="2015/05/22 17:45:00"   # UT string
str(KAT.sidereal_time())


'11:10:47.05'

In [127]:
date="2017/05/22"
time="20:00"
ut_test=UT(date,time)
print ut_test
lst=LST(ut_test)
print SAST(ut_test), ut_test, lst

2017/05/22 18:00:00
2017/05/22 20:00:00 2017/05/22 18:00:00 11:27:50


In [128]:
inp=("dec30, J2000,7:00:00,30:00:00", "dec20, J2000, 7:00:00,20:00:00", "dec0, J2000, 7:00:00,00:00:00"\
     , "dec-20, J2000,7:00:00,-20:00:00", "dec-40, J2000,7:00:00,-40:00:00", "dec-60, J2000,7:00:00,-60:00:00")
for line in inp:
  if line[0] != "#":
    s=line.split(",") # split the line into strings
    source=s[0]
# next item is 'radec...'    
    ra=s[2]
    dec=s[3]
    dbline='"'+source+",f|J|Q,"+ra+","+dec+",,2000"+'"'
#    print dbline
    p=readdb(dbline)
#    p=FixedBody()
#    p._ra,p._dec=ra,dec
#    p._epoch='2000'
    p.compute(KAT)
    if p.neverup == True:
       print "%-11s  %12s %12s           Never Rises           " %(source,p.ra,p.dec)
    if p.circumpolar == True:
       transit=str(KAT.next_transit(p))[-8:]
       print "%-11s  %12s %12s          %9s  Circumpolar "  %(source,p.ra,p.dec,transit)
    if p.neverup == False and p.circumpolar == False:  # Source rises and sets
#      print source, p.ra, p.dec, p.alt, p.az,
      rises=str(KAT.next_rising(p))[-8:]
      transit=str(KAT.next_transit(p))[-8:]
      sets=str(KAT.next_setting(p))[-8:]
      uptime=((KAT.next_setting(p)-KAT.next_rising(p))%1)*24  # 
#calculation is in days, so make modulo 1 day, times 24 for hours.
      print "%-12s %12s %12s %-9s %-9s %-9s  %5.2f " % (source,p.ra,p.dec,rises,transit,sets,uptime)
#    for n in range(3,len(s)):    #comments at the end of the input line
#         print s[n],
#    print ""


dec30          7:01:04.40   29:58:25.5 11:03:43  13:30:01  15:56:20    4.88 
dec20          7:00:59.79   19:58:23.1 10:09:27  13:29:57  16:50:26    6.68 
dec0           7:00:51.84   -0:01:41.6 09:04:15  13:29:49  17:55:23    8.85 
dec-20         7:00:43.76  -20:01:45.9 08:18:17  13:29:41  18:45:01   10.45 
dec-40         7:00:33.05  -40:01:49.1 07:35:09  13:29:30  19:27:47   11.88 
dec-60         7:00:12.66  -60:01:50.8 06:36:08  13:29:10  20:26:07   13.83 


In [129]:
print p.alt,p.az

20:00:00.0 148:52:51.2


In [130]:
KAT

<ephem.Observer date='2017/5/22 18:00:00' epoch='2017/5/20 00:00:00' lon='21:24:38.5' lat='-30:43:17.3' elevation=1038.0m horizon=20:00:00.0 temp=14.0C pressure=0.0mBar>

In [131]:
ut=today+" 13:47:20"
print LST(ut)
print JD(ut)

6:54:46.
2457891.07454


In [132]:
KAT.date=now()
print LST(KAT.date)

2:30:47.


In [133]:
str(localtime(now()))

'2017-05-17 11:24:04.000002'

In [134]:
now()

42870.89171296296

In [135]:
JD(ut)-now()-JD("1899/12/31 12:00:00")

0.18282407382503152

In [136]:
JD("2015/05/06 12:00:00")

2457149.0

In [137]:
print JD("1899/12/31 12:00:00"), LST("1899/12/31 12:00:00")

2415020.0 20:04:25


In [138]:
str(now()) # UT time now

'2017/5/17 09:24:04'

In [139]:
def tomorrow():
    tod=datetime.datetime.today().date() #today
    tom=tod+datetime.timedelta(1) # tomorrow
    toms=tom.strftime("%Y/%m/%d")
    return toms

In [140]:
KAT.horizon='00:00' # NOT 20deg like regular sources
sun.compute(KAT)
sunrise=KAT.next_rising(sun)
sunset=KAT.next_setting(sun)
print "Sunset ",localtime(sunset)
print "Sunrise",localtime(sunrise)
#

Sunset  1899-12-31 19:07:46.000006
Sunrise 1900-01-01 05:07:52


In [141]:
print tomorrow()
tom=tomorrow()
print SAST(str(KAT.next_rising(sun,start=tomorrow())))

2017/05/18
2017/05/18 07:18:17


In [142]:

print "LST at sunset ", LST(str(sunset))
print "LST at sunrise", LST(str(KAT.next_rising(sun,start=tom)))
print "SAST Sunset   ", SAST(str(sunset))
print "SAST Sunrise  ", SAST(str(KAT.next_rising(sun,start=tom)))

LST at sunset  1:43:05.
LST at sunrise 22:28:16
SAST Sunset   

ValueError: year=1899 is before 1900; the datetime strftime() methods require year >= 1900